In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import mss.tools
import time
import uuid
import keyboard
import math
import pyautogui
import time
import mss.tools
import uuid
import keyboard
from pynput import mouse

time.sleep(5)

def screenshot(folder):
    with mss.mss() as sct:
        monitor_info = sct.monitors[0]
        top_margin = (monitor_info["height"] // 4) + 50
        monitor = {"top": monitor_info["top"] + top_margin, "left": monitor_info["left"] + 55, "width": monitor_info["width"] - 120, "height": (monitor_info["height"] * 2 // 3)-450}
        output = f"croppeddataingrid/{folder}/{str(uuid.uuid4())}.png".format(**monitor)
        sct_img = sct.grab(monitor)
        mss.tools.to_png(sct_img.rgb, sct_img.size, output=output)
    return output
        

def on_keyboard_event(event):
    print(f"{event.name}")
    return screenshot(event.name)
    
def on_mouse_event(x, y, button, pressed):    
    print(f"{button.name}")
    return screenshot('forward')

keyboard.hook(on_keyboard_event)

with mouse.Listener(on_click=on_mouse_event) as listener:
    print("Press keys or click the mouse (Press 'Esc' to exit)")
    keyboard.wait('space')  # Wait for the 'esc' key to exit the program
    listener.stop()  # Stop the mouse listener
    keyboard.unhook_all()  # Unhook the keyboard eventsdq

In [17]:
import splitfolders
input_folder = './DLIngrid'
output_folder = './datasetsingrid'
 
#split the images into 80% training and 20% testing sets
splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.2))

Copying files: 1743 files [00:37, 45.93 files/s] 


In [18]:
from keras.preprocessing.image import ImageDataGenerator

# Define the data generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   #horizontal_flip=True
                                   )

# Load and augment the training data
train_generator = train_datagen.flow_from_directory(
    './datasetsingrid/train',
    target_size=(224, 224),  # Adjust the target size based on your model
    batch_size=32,
    class_mode='categorical'
)


Found 1393 images belonging to 3 classes.


In [19]:
from keras.applications import ResNet50
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

# Build your model on top of the ResNet base
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(3, activation='softmax'))  # 3 classes: forward, d, q

# Freeze the layers of the ResNet base
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [20]:
model.fit(
    train_generator,
    epochs=10,  # Adjust the number of epochs based on your dataset and training needs
)

Epoch 1/10
44/44 [==============================] - 234s 5s/step - loss: 0.9960 - accuracy: 0.5485
Epoch 2/10
44/44 [==============================] - 162s 4s/step - loss: 0.9861 - accuracy: 0.5492
Epoch 3/10
44/44 [==============================] - 162s 4s/step - loss: 0.9899 - accuracy: 0.5485
Epoch 4/10
44/44 [==============================] - 121s 3s/step - loss: 0.9758 - accuracy: 0.5485
Epoch 5/10
44/44 [==============================] - 92s 2s/step - loss: 0.9761 - accuracy: 0.5485
Epoch 6/10
44/44 [==============================] - 167s 4s/step - loss: 0.9743 - accuracy: 0.5499
Epoch 7/10
44/44 [==============================] - 130s 3s/step - loss: 0.9674 - accuracy: 0.5528
Epoch 8/10
44/44 [==============================] - 74s 2s/step - loss: 0.9821 - accuracy: 0.5327
Epoch 9/10
44/44 [==============================] - 78s 2s/step - loss: 0.9731 - accuracy: 0.5556
Epoch 10/10
44/44 [==============================] - 84s 2s/step - loss: 0.9697 - accuracy: 0.5535


In [ ]:
# Load and preprocess validation data
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    'datasetsingrid/val',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Evaluate the model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation Accuracy: {accuracy * 100:.2f}%')


In [22]:
from keras.models import load_model, save_model

# Assuming your model is already trained and stored in the 'model' variable
# Train your model ...

# Save the model to an HDF5 file
model.save('my_model.h5')

# To load the model back later, you can use the following:
# loaded_model = load_model('your_model.h5')


c:\Users\dinab\Self Driving Mario Cart\Self-Driving-Mario-Kart\mario_env\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [25]:
print(train_generator.class_indices)

{'d': 0, 'forward': 1, 'q': 2}


In [32]:
import cv2
import pyautogui
import numpy as np
from keras.models import load_model
import keyboard

# Load your pre-trained model
model = load_model('my_model.h5')

# Function to preprocess the image
def preprocess_image(img):
    # Resize the image to match the input size of your model
    img = cv2.resize(img, (224, 224))
    # Normalize pixel values to be in the range [0, 1]
    img = img / 255.0
    # Expand dimensions to match the input shape expected by the model
    img = np.expand_dims(img, axis=0)
    return img

# Start the game (replace 'start_game_command' with the actual command to start your game)
# This command could be launching the emulator or opening the game window
start_game_command = 'start_game_command'
# Uncomment the line below if you need to launch the game
os.system(start_game_command)

isTrue = True
# Main loop
while isTrue:
    # Capture the game screen
    screenshot = pyautogui.screenshot()
    frame = np.array(screenshot)
    
    # Convert RGB to BGR (OpenCV uses BGR)
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # Preprocess the image
    processed_image = preprocess_image(frame)
    
    # Make a prediction using your model
    prediction = model.predict(processed_image)
    
    # Determine the action based on the prediction
    action_index = np.argmax(prediction)

    print(action_index)
    # Perform the corresponding action
    if action_index == 0:  # Replace 0 with the index for 'forward' class
        # Press the mouse (assuming it's equivalent to 'forward' action in your game)
        keyboard.press('q')
        pyautogui.mouseDown()
        time.sleep(0.15)
        pyautogui.mouseUp()
        keyboard.release('q')
    elif action_index == 2:  # Replace 2 with the index for 'q' class
        # Press 'q' key
        keyboard.press('d')
        pyautogui.mouseDown()
        time.sleep(0.15)
        pyautogui.mouseUp()
        keyboard.redlease('d')
    
    if keyboard.is_pressed("space"):
        isTrue = False

1/1 [==============================] - 2s 2s/step
0
1/1 [==============================] - 0s 196ms/step
0
1/1 [==============================] - 0s 200ms/step
0
1/1 [==============================] - 0s 200ms/step
0
1/1 [==============================] - 0s 271ms/step
0
1/1 [==============================] - 0s 294ms/step
0
1/1 [==============================] - 0s 283ms/step
0
1/1 [==============================] - 0s 279ms/step
0
1/1 [==============================] - 0s 257ms/step
0
1/1 [==============================] - 0s 273ms/step
0
1/1 [==============================] - 0s 221ms/step
0
1/1 [==============================] - 0s 218ms/step
0
1/1 [==============================] - 0s 204ms/step
0
1/1 [==============================] - 0s 235ms/step
0
1/1 [==============================] - 0s 209ms/step
0
1/1 [==============================] - 0s 252ms/step
0
1/1 [==============================] - 0s 323ms/step
0
1/1 [==============================] - 0s 297ms/step
0
1/1 [========